In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import os

from absl import flags
from absl.testing import parameterized
import numpy as np
import protein_dataset
import utils
import tensorflow as tf

FLAGS = flags.FLAGS

In [8]:
import inference
model_path = 'cached_models/noxpd2_cnn_swissprot_go_random_swiss-cnn_for_swissprot_go_random-13703706'
model_name = "GO"
inferrer = inference.Inferrer(
        savedmodel_dir_path=model_path,
        use_tqdm= True,
        batch_size=16,
        activation_type="pooled_representation"
)
label_vocab = inferrer.get_variable('label_vocab:0').astype(str)

2023-08-27 22:14:18.365356: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Found device 0 with properties: 
name: Tesla T4 major: 7 minor: 5 memoryClockRate(GHz): 1.59
pciBusID: 0001:00:00.0
2023-08-27 22:14:18.365571: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcudart.so.10.0'; dlerror: libcudart.so.10.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/intel/compilers_and_libraries_2018.1.163/linux/tbb/lib/intel64_lin/gcc4.7:/opt/intel/compilers_and_libraries_2018.1.163/linux/compiler/lib/intel64_lin:/opt/intel/compilers_and_libraries_2018.1.163/linux/mkl/lib/intel64_lin::/usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64/
2023-08-27 22:14:18.365667: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcublas.so.10.0'; dlerror: libcublas.so.10.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/intel/compi

In [10]:
with tf.Session(graph=inferrer._graph) as sess:
    tf.saved_model.loader.load(sess, [tf.saved_model.tag_constants.SERVING],
                                model_path)
    label_vocab = tf.get_default_graph().get_tensor_by_name('label_vocab:0')
    decision_threshold = sess.run(
        tf.get_default_graph().get_tensor_by_name('decision_threshold:0'))

2023-08-27 22:16:26.454839: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1180] Device interconnect StreamExecutor with strength 1 edge matrix:
2023-08-27 22:16:26.454869: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1186]      


KeyError: "The name 'decision_threshold:0' refers to a Tensor which does not exist. The operation, 'decision_threshold', does not exist in the graph."

In [11]:
a=tf.get_default_graph()

In [12]:
a.get_operations

<bound method Graph.get_operations of <tensorflow.python.framework.ops.Graph object at 0x7f3a76f280d0>>

In [2]:
    

def _numpy_one_hot(x, depth):
  """Convert numpy array of indexes into a full one-hot.

  Args:
    x: np.array.
    depth: int. maximum index in array (depth of one-hot output).

  Returns:
    np.array.
  """
  return np.eye(depth)[x]


def _dataset_iterator_to_list(itr, session):
  """Convert tf.data.Dataset iterator to a python list.

  Args:
    itr: tf.data.Dataset iterator.
    session: tf.Session.

  Returns:
    list.
  """
  actual_examples = []
  while True:
    try:
      actual_examples.append(session.run(itr.get_next()))
    except tf.errors.OutOfRangeError:
      break
  return actual_examples

test_data_directory = './testdata'
batch_size = 3

label_vocab_array = [
    'EMBL:AE017224', 'RefSeq:WP_002966386.1', 'ProteinModelPortal:P0CB34',
    'SMR:P0CB34', 'EnsemblBacteria:AAX75635', 'GeneID:29595679',
    'KEGG:bmb:BruAb2_0191', 'HOGENOM:HOG000133897', 'KO:K04078',
    'OMA:PGRIDDN', 'Proteomes:UP000000540', 'GO:GO:0005737',
    'GO:GO:0005524', 'GO:GO:0006457', 'CDD:cd00320', 'Gene3D:2.30.33.40',
    'HAMAP:MF_00580', 'InterPro:IPR020818', 'InterPro:IPR037124',
    'InterPro:IPR018369', 'InterPro:IPR011032', 'PANTHER:PTHR10772',
    'Pfam:PF00166', 'PRINTS:PR00297', 'SMART:SM00883', 'SUPFAM:SSF50129',
    'PROSITE:PS00681'
]
with tf.Graph().as_default():
    sess = tf.Session()
    dataset = protein_dataset.non_batched_dataset(
        # Dev fold instead of train fold because the train fold is repeated.
        train_dev_or_test=protein_dataset.DEV_FOLD,
        label_vocab=label_vocab_array,
        data_root_dir=test_data_directory)
    example_itr = dataset.make_initializable_iterator()

    sess.run(tf.tables_initializer())
    sess.run(tf.global_variables_initializer())
    sess.run(example_itr.initializer)

# Compute actual output
actual_examples = _dataset_iterator_to_list(example_itr, sess)

expected_length = 4

# Compute expected values
expected_sequence = 'MADIKFRPLHDRVVVRRVESEAKTAGGIIIPDTAKEKPQEGEVVAAGAGARDEAGKLVPLDVKAGDRVLFGKWSGTEVKIGGEDLLIMKESDILGIVG'
expected_sequence_indexes = [
    utils.AMINO_ACID_VOCABULARY.index(x) for x in expected_sequence
]
expected_sequence_one_hot = _numpy_one_hot(
    expected_sequence_indexes, depth=len(utils.AMINO_ACID_VOCABULARY))
# Because the label vocab is exactly the labels in the first example, we
# just get range(len(label_vocab_array))
expected_label_indexes = range(len(label_vocab_array))
expected_id = b'P0CB34'


W0812 22:36:09.294070 140338839369344 module_wrapper.py:139] From /tmp/ipykernel_1586523/289269768.py:47: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

2023-08-12 22:36:09.295550: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2023-08-12 22:36:09.400302: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Found device 0 with properties: 
name: Tesla T4 major: 7 minor: 5 memoryClockRate(GHz): 1.59
pciBusID: 0001:00:00.0
2023-08-12 22:36:09.400441: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcudart.so.10.0'; dlerror: libcudart.so.10.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/intel/compilers_and_libraries_2018.1.163/linux/tbb/lib/intel64_lin/gcc4.7:/opt/intel/compilers_and_libraries_2018.1.163/linux/compiler/lib/intel64_lin:/opt/intel/compilers_and_libraries_2018.1.163/linux/mkl/lib/intel64_lin

In [2]:
inferrer = inference.Inferrer(
        savedmodel_dir_path=model_path,
        use_tqdm= True,
        batch_size=16,
        activation_type="pooled_representation"
)
output_path = os.path.join(output_dir,f'{model_name}_label_vocab.json')
label_vocab = inferrer.get_variable(vocab_variable_name).astype(str)

NameError: name 'inference' is not defined

In [3]:
from protein_dataset import *
from protein_dataset import _add_sequence_length,_is_sequence_short_enough_for_training

def non_batched_dataset_(train_dev_or_test,
                        label_vocab,
                        data_root_dir=DATA_ROOT_DIR):
  """Constructs a dataset of examples.

  Args:
    train_dev_or_test: one of _DEV_FOLD_VALUES. The source examples to load into
      a dataset.
    label_vocab: list of string.
    data_root_dir: path to tfrecord examples.

  Returns:
    tf.data.Dataset, where each example is of form
    {
        SEQUENCE_KEY: one-hot of amino acid characters
        SEQUENCE_LENGTH_KEY: length of sequence
        SEQUENCE_ID_KEY: unique identifier for protein
        LABEL_KEY: rank-1 tensor of integer labels from label_vocab,
    }
  """
  if train_dev_or_test not in DATA_FOLD_VALUES:
    raise ValueError(('Only train, dev, test and * are supported datasets.'
                      ' Received {}.').format(train_dev_or_test))
  dataset_files = [
      os.path.join(data_root_dir, f)
      for f in tf.gfile.ListDirectory(data_root_dir)
      if train_dev_or_test in f and ".tfrecord" in f
  ]

  tfrecord_dataset = tf.data.TFRecordDataset(dataset_files)

  dataset = tfrecord_dataset.map(lambda record: tf.io.parse_single_example(  # pylint: disable=g-long-lambda
      record, DATASET_FEATURES))
  dataset = dataset.map(_add_sequence_length)
  dataset = dataset.filter(_is_sequence_short_enough_for_training)

  amino_acid_table = contrib_lookup.index_table_from_tensor(
      utils.AMINO_ACID_VOCABULARY,
      default_value=len(utils.AMINO_ACID_VOCABULARY))
  protein_class_table = contrib_lookup.index_table_from_tensor(
      mapping=label_vocab)

  #dataset = dataset.map(lambda ex: _map_sequence_to_ints(ex, amino_acid_table))
  #dataset = dataset.map(lambda ex: _map_labels_to_ints(ex, protein_class_table))
  #dataset = dataset.map(_to_one_hot_sequence)

  if train_dev_or_test == TRAIN_FOLD:
    dataset = dataset.repeat()

  dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

  return dataset


      

def _numpy_one_hot(x, depth):
  """Convert numpy array of indexes into a full one-hot.

  Args:
    x: np.array.
    depth: int. maximum index in array (depth of one-hot output).

  Returns:
    np.array.
  """
  return np.eye(depth)[x]


def _dataset_iterator_to_list(itr, session):
  """Convert tf.data.Dataset iterator to a python list.

  Args:
    itr: tf.data.Dataset iterator.
    session: tf.Session.

  Returns:
    list.
  """
  actual_examples = []
  while True:
    try:
      actual_examples.append(session.run(itr.get_next()))
    except tf.errors.OutOfRangeError:
      break
  return actual_examples

test_data_directory = './testdata'
batch_size = 3

label_vocab_array = [
    'EMBL:AE017224', 'RefSeq:WP_002966386.1', 'ProteinModelPortal:P0CB34',
    'SMR:P0CB34', 'EnsemblBacteria:AAX75635', 'GeneID:29595679',
    'KEGG:bmb:BruAb2_0191', 'HOGENOM:HOG000133897', 'KO:K04078',
    'OMA:PGRIDDN', 'Proteomes:UP000000540', 'GO:GO:0005737',
    'GO:GO:0005524', 'GO:GO:0006457', 'CDD:cd00320', 'Gene3D:2.30.33.40',
    'HAMAP:MF_00580', 'InterPro:IPR020818', 'InterPro:IPR037124',
    'InterPro:IPR018369', 'InterPro:IPR011032', 'PANTHER:PTHR10772',
    'Pfam:PF00166', 'PRINTS:PR00297', 'SMART:SM00883', 'SUPFAM:SSF50129',
    'PROSITE:PS00681'
]
with tf.Graph().as_default():
    sess = tf.Session()
    dataset = non_batched_dataset_(
        # Dev fold instead of train fold because the train fold is repeated.
        train_dev_or_test=protein_dataset.DEV_FOLD,
        label_vocab=label_vocab_array,
        data_root_dir=test_data_directory)
    example_itr = dataset.make_initializable_iterator()

    sess.run(tf.tables_initializer())
    sess.run(tf.global_variables_initializer())
    sess.run(example_itr.initializer)

# Compute actual output
actual_examples_decoded = _dataset_iterator_to_list(example_itr, sess)



2023-08-12 22:36:10.165413: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1180] Device interconnect StreamExecutor with strength 1 edge matrix:
2023-08-12 22:36:10.165443: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1186]      


In [4]:
actual_examples_decoded

[{'id': b'P0CB34',
  'label': array([b'EMBL:AE017224', b'RefSeq:WP_002966386.1',
         b'ProteinModelPortal:P0CB34', b'SMR:P0CB34',
         b'EnsemblBacteria:AAX75635', b'GeneID:29595679',
         b'KEGG:bmb:BruAb2_0191', b'HOGENOM:HOG000133897', b'KO:K04078',
         b'OMA:PGRIDDN', b'Proteomes:UP000000540', b'GO:GO:0005737',
         b'GO:GO:0005524', b'GO:GO:0006457', b'CDD:cd00320',
         b'Gene3D:2.30.33.40', b'HAMAP:MF_00580', b'InterPro:IPR020818',
         b'InterPro:IPR037124', b'InterPro:IPR018369',
         b'InterPro:IPR011032', b'PANTHER:PTHR10772', b'Pfam:PF00166',
         b'PRINTS:PR00297', b'SMART:SM00883', b'SUPFAM:SSF50129',
         b'PROSITE:PS00681'], dtype=object),
  'sequence': b'MADIKFRPLHDRVVVRRVESEAKTAGGIIIPDTAKEKPQEGEVVAAGAGARDEAGKLVPLDVKAGDRVLFGKWSGTEVKIGGEDLLIMKESDILGIVG',
  'sequence_length': 98},
 {'id': b'Q6ARV5',
  'label': array([b'EMBL:CR522870', b'RefSeq:WP_011187436.1',
         b'ProteinModelPortal:Q6ARV5', b'SMR:Q6ARV5',
         b'STRIN

In [5]:
import sys
HOME = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
protein_functions_path = os.path.join(HOME,'ProteinFunctions')
sys.path.append(protein_functions_path)

In [6]:
from src.data.datasets import ProteinDataset

PD = ProteinDataset(data_path='../ProteinFunctions/data/swissprot/proteinfer_splits/random/train_GO.fasta',
                    sequence_vocabulary_path='../ProteinFunctions/data/vocabularies/amino_acid_vocab.json',
                    label_vocabulary_path='../ProteinFunctions/data/vocabularies/GO_label_vocab.json')

In [7]:
for idx in range(len(actual_examples_decoded)):
    example = actual_examples_decoded[idx]
    #print(example)
    go_labels = [example['id'].decode()]+['GO:'+i.decode().split('GO:')[-1] for i in example['label'] if 'GO:' in i.decode()]

    PD_seq,PD_lab = PD.process_example(example['sequence'].decode(),go_labels)

    print('decoded labels:',[PD.int2label[i] for i in np.where(PD_lab==1)[0]])
    print('actual labels:',go_labels)
    print((actual_examples[idx]['sequence']== PD_seq.numpy()).all())

decoded labels: ['GO:0005524', 'GO:0005737', 'GO:0006457']
actual labels: ['P0CB34', 'GO:0005737', 'GO:0005524', 'GO:0006457']
True
decoded labels: ['GO:0005524', 'GO:0005737', 'GO:0006457']
actual labels: ['Q6ARV5', 'GO:0005737', 'GO:0005524', 'GO:0006457']
True
decoded labels: ['GO:0005524', 'GO:0005737', 'GO:0006457']
actual labels: ['B8FNT6', 'GO:0005737', 'GO:0005524', 'GO:0006457']
True
decoded labels: ['GO:0005524', 'GO:0005737', 'GO:0006457']
actual labels: ['A9MDV2', 'GO:0005737', 'GO:0005524', 'GO:0006457']
True


In [8]:
PD_seq.shape

torch.Size([98, 20])

In [21]:
import torch


torch.Size([100, 20])

In [26]:
[PD.int2label[i] for i in np.where(PD_lab==1)[0]]

['GO:0005524', 'GO:0005737', 'GO:0006457']

In [31]:
np.testing.assert_equal(actual_examples[0][protein_dataset.SEQUENCE_KEY],
                            expected_sequence_one_hot)

In [49]:
actual_examples[2]['sequence'].sum(axis=1)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)

In [51]:
    actual_sequence_batch_shape = actual_examples[0][
        protein_dataset.SEQUENCE_KEY].shape
actual_sequence_batch_shape 

(98, 20)

In [56]:
actual_examples[0]

{'id': b'P0CB34',
 'label': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26]),
 'sequence': array([[0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 'sequence_length': 98}

In [99]:

  def test_padded_dataset(self):
    # Set up test data.
    test_data_directory = os.path.join(
        FLAGS.test_srcdir,
        './testdata'
    )

    label_vocab_array = ['EMBL:AE017224']

    batch_size = 3

    with tf.Graph().as_default():
      sess = tf.Session()
      non_padded_dataset = protein_dataset.non_batched_dataset(
          # Dev fold instead of train fold because the train fold is repeated.
          train_dev_or_test=protein_dataset.DEV_FOLD,
          label_vocab=label_vocab_array,
          data_root_dir=test_data_directory)
      batched_dataset = protein_dataset.batched_dataset(
          non_padded_dataset, batch_size=batch_size, bucket_boundaries=[11000])
      batch_itr = batched_dataset.make_initializable_iterator()
      non_padded_itr = non_padded_dataset.make_initializable_iterator()

      sess.run(tf.tables_initializer())
      sess.run(tf.global_variables_initializer())
      sess.run(batch_itr.initializer)

    # Compute actual output
    actual_examples = _dataset_iterator_to_list(batch_itr, sess)
    actual_examples_non_padded = _dataset_iterator_to_list(non_padded_itr, sess)

    # Examine correctness of first element.
    actual_sequence_batch_shape = actual_examples[0][
        protein_dataset.SEQUENCE_KEY].shape
    expected_longest_sequence_len_in_first_batch = 98
    expected_first_batch_sequence_shape = (
        batch_size, expected_longest_sequence_len_in_first_batch,
        len(utils.AMINO_ACID_VOCABULARY))
    self.assertEqual(actual_sequence_batch_shape,
                     expected_first_batch_sequence_shape)

    actual_label_batch_shape = actual_examples[0][
        protein_dataset.LABEL_KEY].shape
    # Because the label vocab contains the labels in the first example, we
    # get len(label_vocab_array) as the number of labels.
    expected_batch_label_shape = (batch_size, len(label_vocab_array))
    self.assertEqual(actual_label_batch_shape, expected_batch_label_shape)

  def test_yield_examples(self):
    path = os.path.join(
        FLAGS.test_srcdir,
        './testdata/train*.tfrecord'
    )
    actual_examples = list(protein_dataset.yield_examples(path))
    expected_length = 4
    self.assertLen(actual_examples, expected_length)
    self.assertEqual(
        set(actual_examples[0].keys()),
        set([
            protein_dataset.SEQUENCE_KEY, protein_dataset.SEQUENCE_ID_KEY,
            protein_dataset.LABEL_KEY
        ]))

main()

NameError: name 'main' is not defined

In [97]:
actual_examples_non_padded

NameError: name 'actual_examples_non_padded' is not defined

In [66]:
actual_examples

[{'id': b'P0CB34',
  'label': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
         17, 18, 19, 20, 21, 22, 23, 24, 25, 26]),
  'sequence': array([[0., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         [0., 0., 1., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 1., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
  'sequence_length': 98},
 {'id': b'Q6ARV5',
  'label': array([ 7,  8,  9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24,
         25, 26]),
  'sequence': array([[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
  'sequence_length': 95},
 {'id': b'B8FNT6',
  'label': array([ 7,  8, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 2

In [74]:
# coding=utf-8
# Copyright 2020 The Google Research Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import os

from absl import flags
from absl.testing import parameterized
import numpy as np
import protein_dataset
import utils
import tensorflow as tf

FLAGS = flags.FLAGS


def _numpy_one_hot(x, depth):
  """Convert numpy array of indexes into a full one-hot.

  Args:
    x: np.array.
    depth: int. maximum index in array (depth of one-hot output).

  Returns:
    np.array.
  """
  return np.eye(depth)[x]


def _dataset_iterator_to_list(itr, session):
  """Convert tf.data.Dataset iterator to a python list.

  Args:
    itr: tf.data.Dataset iterator.
    session: tf.Session.

  Returns:
    list.
  """
  actual_examples = []
  while True:
    try:
      actual_examples.append(session.run(itr.get_next()))
    except tf.errors.OutOfRangeError:
      break
  return actual_examples


class ProteinDatasetTest(parameterized.TestCase):

  def test_non_padded_dataset(self):
    # Set up test data.
    test_data_directory = os.path.join(
        FLAGS.test_srcdir,
        './testdata'
    )

    label_vocab_array = [
        'EMBL:AE017224', 'RefSeq:WP_002966386.1', 'ProteinModelPortal:P0CB34',
        'SMR:P0CB34', 'EnsemblBacteria:AAX75635', 'GeneID:29595679',
        'KEGG:bmb:BruAb2_0191', 'HOGENOM:HOG000133897', 'KO:K04078',
        'OMA:PGRIDDN', 'Proteomes:UP000000540', 'GO:GO:0005737',
        'GO:GO:0005524', 'GO:GO:0006457', 'CDD:cd00320', 'Gene3D:2.30.33.40',
        'HAMAP:MF_00580', 'InterPro:IPR020818', 'InterPro:IPR037124',
        'InterPro:IPR018369', 'InterPro:IPR011032', 'PANTHER:PTHR10772',
        'Pfam:PF00166', 'PRINTS:PR00297', 'SMART:SM00883', 'SUPFAM:SSF50129',
        'PROSITE:PS00681'
    ]
    with tf.Graph().as_default():
      sess = tf.Session()
      dataset = protein_dataset.non_batched_dataset(
          # Dev fold instead of train fold because the train fold is repeated.
          train_dev_or_test=protein_dataset.DEV_FOLD,
          label_vocab=label_vocab_array,
          data_root_dir=test_data_directory)
      example_itr = dataset.make_initializable_iterator()

      sess.run(tf.tables_initializer())
      sess.run(tf.global_variables_initializer())
      sess.run(example_itr.initializer)

    # Compute actual output
    actual_examples = _dataset_iterator_to_list(example_itr, sess)

    expected_length = 4

    # Compute expected values
    expected_sequence = 'MADIKFRPLHDRVVVRRVESEAKTAGGIIIPDTAKEKPQEGEVVAAGAGARDEAGKLVPLDVKAGDRVLFGKWSGTEVKIGGEDLLIMKESDILGIVG'
    expected_sequence_indexes = [
        utils.AMINO_ACID_VOCABULARY.index(x) for x in expected_sequence
    ]
    expected_sequence_one_hot = _numpy_one_hot(
        expected_sequence_indexes, depth=len(utils.AMINO_ACID_VOCABULARY))
    # Because the label vocab is exactly the labels in the first example, we
    # just get range(len(label_vocab_array))
    expected_label_indexes = range(len(label_vocab_array))
    expected_id = b'P0CB34'

    # Assert values correct
    self.assertLen(actual_examples, expected_length)
    np.testing.assert_equal(actual_examples[0][protein_dataset.SEQUENCE_KEY],
                            expected_sequence_one_hot)
    np.testing.assert_equal(
        actual_examples[0][protein_dataset.SEQUENCE_LENGTH_KEY],
        len(expected_sequence))
    np.testing.assert_equal(actual_examples[0][protein_dataset.LABEL_KEY],
                            expected_label_indexes)
    np.testing.assert_equal(actual_examples[0][protein_dataset.SEQUENCE_ID_KEY],
                            expected_id)

  def test_padded_dataset(self):
    # Set up test data.
    test_data_directory = os.path.join(
        FLAGS.test_srcdir,
        './testdata'
    )

    label_vocab_array = ['EMBL:AE017224']

    batch_size = 2

    with tf.Graph().as_default():
      sess = tf.Session()
      non_padded_dataset = protein_dataset.non_batched_dataset(
          # Dev fold instead of train fold because the train fold is repeated.
          train_dev_or_test=protein_dataset.DEV_FOLD,
          label_vocab=label_vocab_array,
          data_root_dir=test_data_directory)
      batched_dataset = protein_dataset.batched_dataset(
          non_padded_dataset, batch_size=batch_size, bucket_boundaries=[11000])
      batch_itr = batched_dataset.make_initializable_iterator()

      sess.run(tf.tables_initializer())
      sess.run(tf.global_variables_initializer())
      sess.run(batch_itr.initializer)

    # Compute actual output
    actual_examples = _dataset_iterator_to_list(batch_itr, sess)

    # Examine correctness of first element.
    actual_sequence_batch_shape = actual_examples[0][
        protein_dataset.SEQUENCE_KEY].shape
    expected_longest_sequence_len_in_first_batch = 98
    expected_first_batch_sequence_shape = (
        batch_size, expected_longest_sequence_len_in_first_batch,
        len(utils.AMINO_ACID_VOCABULARY))
    self.assertEqual(actual_sequence_batch_shape,
                     expected_first_batch_sequence_shape)

    actual_label_batch_shape = actual_examples[0][
        protein_dataset.LABEL_KEY].shape
    # Because the label vocab contains the labels in the first example, we
    # get len(label_vocab_array) as the number of labels.
    expected_batch_label_shape = (batch_size, len(label_vocab_array))
    self.assertEqual(actual_label_batch_shape, expected_batch_label_shape)

  def test_yield_examples(self):
    path = os.path.join(
        FLAGS.test_srcdir,
        './testdata/train*.tfrecord'
    )
    actual_examples = list(protein_dataset.yield_examples(path))
    expected_length = 4
    self.assertLen(actual_examples, expected_length)
    self.assertEqual(
        set(actual_examples[0].keys()),
        set([
            protein_dataset.SEQUENCE_KEY, protein_dataset.SEQUENCE_ID_KEY,
            protein_dataset.LABEL_KEY
        ]))


In [75]:
actual_examples[1]['sequence'].sum(axis=1)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)

In [77]:
len(actual_examples)

4

In [105]:
batch_size = 2
with tf.Graph().as_default():
    sess = tf.Session()
    non_padded_dataset = protein_dataset.non_batched_dataset(
        # Dev fold instead of train fold because the train fold is repeated.
        train_dev_or_test=protein_dataset.DEV_FOLD,
        label_vocab=label_vocab_array,
        data_root_dir=test_data_directory)
    batched_dataset = protein_dataset.batched_dataset(
        non_padded_dataset, batch_size=batch_size, bucket_boundaries=[11000])
    batch_itr = batched_dataset.make_initializable_iterator()
    non_batch_itr = non_padded_dataset.make_initializable_iterator()

    sess.run(tf.tables_initializer())
    sess.run(tf.global_variables_initializer())
    sess.run(batch_itr.initializer)

actual_examples = _dataset_iterator_to_list(batch_itr, sess)
#actual_examples_non_padedd = _dataset_iterator_to_list(non_batch_itr, sess)

2023-08-12 17:07:08.184236: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1180] Device interconnect StreamExecutor with strength 1 edge matrix:
2023-08-12 17:07:08.184267: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1186]      


In [108]:
non_batch_itr[0]

TypeError: 'Iterator' object is not subscriptable

In [88]:
actual_examples[0]['sequence'][1].sum(axis=1)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0.], dtype=float32)

In [90]:
actual_examples[0]['label'][1].sum(axis=1)

AxisError: axis 1 is out of bounds for array of dimension 1

In [93]:
actual_examples[0]['label']

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
        16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26],
       [ 7,  8,  9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23,
        24, 25, 26, -1, -1, -1, -1, -1, -1, -1, -1]])

In [23]:
import tensorflow_hub as hub
model_path = 'cached_models/noxpd2_cnn_swissprot_go_random_swiss-cnn_for_swissprot_go_random-13703706'

module_spec=hub.saved_model_module.create_module_spec_from_saved_model(
    model_path)

tags = [tf.saved_model.tag_constants.SERVING]
name_scope='inferrer'
module = hub.Module(module_spec, trainable=True, tags=tags, name=name_scope)

with tf.Session() as sess:


    # Initialize the variables
    sess.run([
          tf.initializers.global_variables(),
          tf.initializers.local_variables(),
          tf.initializers.tables_initializer(),
      ])
    
    # Fetch the trainable variables (weights & biases)
    trainable_vars = tf.trainable_variables()
    
    # Using the session, evaluate each variable to get its value (numpy array)
    trainable_var_values = sess.run(trainable_vars)
    
    # Store the variable names and their values in a dictionary
    #weights_dict = {var.name: value for var, value in zip(trainable_vars, trainable_var_values)}


2023-08-14 12:17:11.572624: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1180] Device interconnect StreamExecutor with strength 1 edge matrix:
W0814 12:17:11.572881 140540827387520 module_wrapper.py:139] From /tmp/ipykernel_3729272/42762710.py:16: The name tf.initializers.global_variables is deprecated. Please use tf.compat.v1.initializers.global_variables instead.

2023-08-14 12:17:11.572655: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1186]      
W0814 12:17:11.575455 140540827387520 module_wrapper.py:139] From /tmp/ipykernel_3729272/42762710.py:17: The name tf.initializers.local_variables is deprecated. Please use tf.compat.v1.initializers.local_variables instead.

W0814 12:17:11.576180 140540827387520 module_wrapper.py:139] From /tmp/ipykernel_3729272/42762710.py:18: The name tf.initializers.tables_initializer is deprecated. Please use tf.compat.v1.initializers.tables_initializer instead.



In [3]:

import inference
model_path = 'cached_models/noxpd2_cnn_swissprot_go_random_swiss-cnn_for_swissprot_go_random-13703706'

inferrer = inference.Inferrer(
        savedmodel_dir_path=model_path,
        use_tqdm= True,
        batch_size=16,
        activation_type="pooled_representation"
)

W0901 05:20:24.977830 139928952129152 deprecation.py:323] From /anaconda/envs/proteinfer/lib/python3.7/site-packages/tensorflow_core/python/ops/ragged/ragged_tensor.py:1586: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [22]:

graph = inferrer._graph

params_of_interest = []
for op in graph.get_operations():
    if ('conv1d' in op.name)&((op.name.endswith('dilation_rate'))|(op.name.endswith('kernel'))):
        params_of_interest.append(op.values()[0].name)


In [23]:

# Initialize all the variables
res = {}
for p in params_of_interest:
    tensor_to_fetch = graph.get_tensor_by_name(p)
    with tf.Session(graph=graph) as sess:
        sess.run(tf.global_variables_initializer())
        val = sess.run(tensor_to_fetch)
    res[p]=val

for k,v in res.items():
    if 'dilation' in k:
        print(k,v)


InvalidArgumentError: You must feed a value for placeholder tensor 'inferrer_apply_pooled_representation/conv1d/kernel' with dtype resource and shape [9,20,1100]
	 [[node inferrer_apply_pooled_representation/conv1d/kernel (defined at /anaconda/envs/proteinfer/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1748) ]]

Original stack trace for 'inferrer_apply_pooled_representation/conv1d/kernel':
  File "/anaconda/envs/proteinfer/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/anaconda/envs/proteinfer/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/anaconda/envs/proteinfer/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/anaconda/envs/proteinfer/lib/python3.7/site-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/anaconda/envs/proteinfer/lib/python3.7/site-packages/ipykernel/kernel

In [25]:
for k,v in res.items():
    if 'dilation' in k:
        print(k,v)

inferrer/conv1d/dilation_rate:0 [1]
inferrer/residual_block_0/conv1d/dilation_rate:0 [1]
inferrer/residual_block_0/conv1d_1/dilation_rate:0 [1]
inferrer/residual_block_1/conv1d/dilation_rate:0 [3]
inferrer/residual_block_1/conv1d_1/dilation_rate:0 [1]
inferrer/residual_block_2/conv1d/dilation_rate:0 [9]
inferrer/residual_block_2/conv1d_1/dilation_rate:0 [1]
inferrer/residual_block_3/conv1d/dilation_rate:0 [27]
inferrer/residual_block_3/conv1d_1/dilation_rate:0 [1]
inferrer/residual_block_4/conv1d/dilation_rate:0 [81]
inferrer/residual_block_4/conv1d_1/dilation_rate:0 [1]


In [16]:
[k for k in res.keys() if 'conv' in k]

['inferrer/conv1d/kernel/Initializer/random_uniform/shape:0',
 'inferrer/conv1d/kernel/Initializer/random_uniform/min:0',
 'inferrer/conv1d/kernel/Initializer/random_uniform/max:0',
 'inferrer/conv1d/kernel/Initializer/random_uniform/RandomUniform:0',
 'inferrer/conv1d/kernel/Initializer/random_uniform/sub:0',
 'inferrer/conv1d/kernel/Initializer/random_uniform/mul:0',
 'inferrer/conv1d/kernel/Initializer/random_uniform:0',
 'inferrer/conv1d/kernel:0',
 'inferrer/conv1d/kernel/Assign:0',
 'inferrer/conv1d/kernel/read:0',
 'inferrer/conv1d/bias/Initializer/zeros/shape_as_tensor:0',
 'inferrer/conv1d/bias/Initializer/zeros/Const:0',
 'inferrer/conv1d/bias/Initializer/zeros:0',
 'inferrer/conv1d/bias:0',
 'inferrer/conv1d/bias/Assign:0',
 'inferrer/conv1d/bias/read:0',
 'inferrer/conv1d/dilation_rate:0',
 'inferrer/conv1d/dilation_rate_1:0',
 'inferrer/conv1d/conv1d/ExpandDims/dim:0']

In [21]:
res

{'UnicodeSplit/UnicodeEncode/UnicodeEncode/RaggedFromTensor/strided_slice/stack:0': array([1], dtype=int32),
 'UnicodeSplit/UnicodeEncode/UnicodeEncode/RaggedFromTensor/strided_slice/stack_1:0': array([2], dtype=int32),
 'UnicodeSplit/UnicodeEncode/UnicodeEncode/RaggedFromTensor/strided_slice/stack_2:0': array([1], dtype=int32)}

In [7]:
writer = tf.summary.FileWriter('./my_graph',graph)

# Close the writer
writer.close()

In [6]:
for op in graph.get_operations():
    if "inferrer" in op.name.lower():  # assuming conv layers have 'conv' in their names
        print(op.name,op.values()[0].name)


inferrer/global_step/Initializer/zeros inferrer/global_step/Initializer/zeros:0
inferrer/global_step inferrer/global_step:0
inferrer/global_step/Assign inferrer/global_step/Assign:0
inferrer/global_step/read inferrer/global_step/read:0
inferrer/batched_one_hot_sequences_placeholder inferrer/batched_one_hot_sequences_placeholder:0
inferrer/sequence_length_placeholder inferrer/sequence_length_placeholder:0
inferrer/Const inferrer/Const:0
inferrer/set_padding_to_sentinel/Shape inferrer/set_padding_to_sentinel/Shape:0
inferrer/set_padding_to_sentinel/strided_slice/stack inferrer/set_padding_to_sentinel/strided_slice/stack:0
inferrer/set_padding_to_sentinel/strided_slice/stack_1 inferrer/set_padding_to_sentinel/strided_slice/stack_1:0
inferrer/set_padding_to_sentinel/strided_slice/stack_2 inferrer/set_padding_to_sentinel/strided_slice/stack_2:0
inferrer/set_padding_to_sentinel/strided_slice inferrer/set_padding_to_sentinel/strided_slice:0
inferrer/set_padding_to_sentinel/Shape_1 inferrer/se

inferrer/residual_block_4/Const inferrer/residual_block_4/Const:0
inferrer/residual_block_4/set_padding_to_sentinel/Shape inferrer/residual_block_4/set_padding_to_sentinel/Shape:0
inferrer/residual_block_4/set_padding_to_sentinel/strided_slice/stack inferrer/residual_block_4/set_padding_to_sentinel/strided_slice/stack:0
inferrer/residual_block_4/set_padding_to_sentinel/strided_slice/stack_1 inferrer/residual_block_4/set_padding_to_sentinel/strided_slice/stack_1:0
inferrer/residual_block_4/set_padding_to_sentinel/strided_slice/stack_2 inferrer/residual_block_4/set_padding_to_sentinel/strided_slice/stack_2:0
inferrer/residual_block_4/set_padding_to_sentinel/strided_slice inferrer/residual_block_4/set_padding_to_sentinel/strided_slice:0
inferrer/residual_block_4/set_padding_to_sentinel/Shape_1 inferrer/residual_block_4/set_padding_to_sentinel/Shape_1:0
inferrer/residual_block_4/set_padding_to_sentinel/strided_slice_1/stack inferrer/residual_block_4/set_padding_to_sentinel/strided_slice_1/

IndexError: tuple index out of range